In [155]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [156]:
# 1. Préparation des données d'entraînement
X = data.drop('Survived', axis=1)
y = data['Survived']

# 2. Entraînement du modèle
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# 3. Préparation des données de test
test_data = pd.read_csv("test.csv")

# 4. Appliquer les mêmes transformations aux données de test
# Créer la feature Family
test_data["Family"] = test_data["SibSp"] + test_data["Parch"]  + 1

# Extraire le titre
test_data["Title"] = test_data["Name"].apply(get_title)
test_data["Title"] = test_data["Title"].replace(rare_titles, "Rare")
   

# Remplir les valeurs manquantes
test_data["Age"] = test_data["Age"].fillna(test_data["Age"].median())
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].median())


# Garder les PassengerId pour la soumission
passenger_ids = test_data["PassengerId"]

# Supprimer les colonnes inutiles
test_data = test_data.drop(columns=["Name", "Ticket", "Cabin", "PassengerId"])

# One-hot encoding
test_data = pd.get_dummies(test_data, columns=["Sex", "Embarked", "Title"], drop_first=True)

# S'assurer que les colonnes de test correspondent à celles d'entraînement
for column in X.columns:
    if column not in test_data.columns:
        test_data[column] = 0
test_data = test_data[X.columns]

# 5. Faire les prédictions
predictions = rf_model.predict(test_data)

# 6. Créer le fichier de soumission
submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': predictions
})






In [157]:
# 1. Charger les données
data = pd.read_csv("train.csv")


# 2. Créer la feature Family
data["Family"] = data["SibSp"] + data["Parch"] + 1

# 3. Extraire le titre depuis le nom
def get_title(name):
    return name.split(",")[1].split(".")[0].strip()

data["Title"] = data["Name"].apply(get_title)

# 4. Regrouper les titres rares
rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major',
               'Rev', 'Sir', 'Jonkheer', 'Dona']
data["Title"] = data["Title"].replace(rare_titles, "Rare")

# 5. Remplir les valeurs manquantes
data["Age"] = data["Age"].fillna(data["Age"].median())# A expliquer pq
data["Embarked"] = data["Embarked"].fillna(data["Embarked"].mode()[0])



# 6. Supprimer la colonne Name (inutile)
data = data.drop(columns=["Name", "Ticket", "Cabin", "PassengerId"])

# 7. Transformer les variables catégorielles en numériques (one-hot)
data = pd.get_dummies(data, columns=["Sex", "Embarked", "Title"], drop_first=True)



#           Évaluation du modèle

In [158]:
# 8. Séparer les données en features X et cible y
X = data.drop("Survived", axis=1)
y = data["Survived"]

# 9. Diviser en jeu d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 10. Créer et entraîner le modèle RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 11. Faire des prédictions sur le test
y_pred = model.predict(X_test)


#12. Sauvegarder le fichier de soumission
submission.to_csv('submission2.csv', index=False)


print("Fichier de soumission créé avec succès !")
print("\nAperçu des prédictions :")
print(submission.head())
print("\nDistribution des prédictions :")
print(submission['Survived'].value_counts(normalize=True).round(3) * 100, "%")

# 13. Afficher les résultats
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Fichier de soumission créé avec succès !

Aperçu des prédictions :
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         1
4          896         0

Distribution des prédictions :
Survived
0    60.0
1    40.0
Name: proportion, dtype: float64 %
Accuracy: 0.8324022346368715

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.86      0.86       105
           1       0.80      0.80      0.80        74

    accuracy                           0.83       179
   macro avg       0.83      0.83      0.83       179
weighted avg       0.83      0.83      0.83       179



#                Analyse de l'importance des caractéristiques (features)

In [ ]:
# Récupérer les importances
feature_importance = model.feature_importances_
feature_names = X.columns

# Créer un DataFrame trié
importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

# Affichage texte
print("\nImportance des caractéristiques :")
print(importances_df)

# Visualisation avec Plotly
import plotly.express as px

fig = px.bar(importances_df.sort_values(by='Importance', ascending=True),  # du plus petit au plus grand pour une barre horizontale
             x='Importance',
             y='Feature',
             orientation='h',
             title="Importance des caractéristiques (Random Forest)",
             labels={"Importance": "Importance", "Feature": "Variable"},
             height=500)
fig.show()



Importance des caractéristiques :
               Feature  Importance
4                 Fare    0.243611
1                  Age    0.208030
12            Title_Mr    0.135273
6             Sex_male    0.118603
0               Pclass    0.077289
5               Family    0.050682
2                SibSp    0.041397
13           Title_Mrs    0.040274
3                Parch    0.023258
9           Title_Miss    0.021278
8           Embarked_S    0.020173
7           Embarked_Q    0.010466
15          Title_Rare    0.009119
14            Title_Ms    0.000322
10          Title_Mlle    0.000140
16  Title_the Countess    0.000067
11           Title_Mme    0.000019


#       Recherche par grille avec validation croisée

<!-- #           Performance Optimization  Grid Search Cross-Validation   -->

In [160]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_model = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=5,                 # 5-fold cross-validation
    scoring='accuracy',   # on optimise l'accuracy
    n_jobs=-1             # utilise tous les CPU disponibles
)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print("Meilleurs paramètres trouvés : ", best_params)


Meilleurs paramètres trouvés :  {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


# 2. Techniques d'optimisation avancées

In [161]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
random_param_dist = {
    'n_estimators': randint(50, 500),                  # nb d’arbres entre 50 et 500
    'max_depth': [None] + list(randint(10, 100).rvs(5)),  # profondeurs entre 10 et 100 + None
    'min_samples_split': randint(2, 20),               # min. d’échantillons pour splitter un noeud
    'max_features': uniform(0.1, 0.9)                  # proportion de features à considérer
}

random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=random_param_dist,
    n_iter=100,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

random_search.fit(X_train, y_train)
print(random_search.best_params_)

best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)






from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

submission.to_csv('submission4.csv', index=False)
print("Fichier de soumission créé avec succès !")
print(submission.head())
print(submission['Survived'].value_counts(normalize=True).round(3) * 100, "%")

{'max_depth': np.int64(67), 'max_features': np.float64(0.21577815032788095), 'min_samples_split': 15, 'n_estimators': 334}
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       105
           1       0.83      0.78      0.81        74

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.84       179
weighted avg       0.84      0.84      0.84       179

Fichier de soumission créé avec succès !
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         1
4          896         0
Survived
0    60.0
1    40.0
Name: proportion, dtype: float64 %


<!-- #  

Précision classe 1 (survivants) : 83 % des prédits comme "survivants" sont corrects.

Rappel classe 1 : le modèle retrouve 73 % des vrais survivants. 

F1-score combine les deux : ici 0.78 → un bon équilibre entre les deux.

Accuracy globale : 83 % → ton modèle prédit correctement 83 passagers sur 100. -->